# Cortes
>$n$: Quantidade de demandas  
>$m$: Quantidade de padrões de corte  

## Variáveis
>$x_j$: Quantidade de bobinas, para $j = 1,\ldots, m$  

## Parâmetros
>$M$: Compromento da bobina matriz.  
>$D_i$: Demanda das bobinas menores, para $i = 1,\ldots, n$  
>$P_{ij}$: Padrões de corte, para $i = 1,\ldots, n$ e $j = 1,\ldots, m$  

## Modelo  

\begin{align*}
    \hbox{min} \ \ 
        & \sum_{i=1}^n x_i \\
        
    \hbox{s.a.} \ \ 
        & \sum_{j=1}^m P_{ij} x_j \leq M   && i = 1,\ldots, n \\
        & P_{ij} \geq 0                    && i = 1,\ldots, n \,;\, j = 1,\ldots, m \\
        & D_i \geq 0                       && i = 1,\ldots, n \\
        & M   \geq 0
\end{align*}

In [3]:
from itertools import product


def cut_patterns(M: int, C: list[int]) -> list[list[int]]:
    corte_minimo = M - min(C)
    cortes = product(*(range(M//i + 1) for i in C))
    x = [i for i in cortes if corte_minimo < sum(a*b for a, b in zip(i, C)) <= M]

    return x

In [112]:
import pandas as pd
from IPython.display import display

def create_table(M: int, C: list[int]):
    patterns = cut_patterns(M, C)
    df = pd.DataFrame(patterns, columns=C)
    df["Resto"] = M - df.mul(C).sum(axis=1)
    display(df)

In [126]:
from pymprog import *

def modelo_cortes(M, C, D):
    P = [*zip(*cut_patterns(M, C))]

    n = len(D)
    m = len(P[0])

    begin("cortes")

    x = var("x", m, int)

    minimize( sum(x) )

    for i in range(n):
        sum(P[i][j] * x[j] for j in range(m)) >= D[i]

    solve()

    res = f"Valor ótimo: {vobj()}\n"
    res += '\n'.join(f"Quantidade usada dp padrão {j:>2}: {x[j].primal}" for j in range(m))

    end()

    return res

In [129]:
import os

for file in os.listdir("./input/"):
    with open(f"input/{file}", "r") as f:
        M = int(f.readline())

        # descarta dado não utilizado
        f.readline()

        C, D = [], []
        for i in f.readlines():
            c, d = map(int, i.split())
            C.append(c)
            D.append(d)

        create_table(M, C)
        res = modelo_cortes(M, C, D)

        with open(f"output/{file}", "w") as f:
            f.write(res)

,21,24,43,79,98,Resto
0,0,0,0,0,2,4
1,0,0,2,0,1,16
2,0,1,0,2,0,18
3,0,1,2,1,0,11
4,0,1,4,0,0,4
5,0,2,1,0,1,11
6,0,3,1,1,0,6
7,0,4,0,0,1,6
8,0,4,2,0,0,18
9,0,5,0,1,0,1


GLPK Simplex Optimizer 5.0
5 rows, 51 columns, 129 non-zeros
      0: obj =   0.000000000e+00 inf =   1.209e+04 (5)
      5: obj =   3.446229167e+03 inf =   0.000e+00 (0)
*     7: obj =   3.338166667e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer 5.0
5 rows, 51 columns, 129 non-zeros
51 integer variables, none of which are binary
Integer optimization begins...
Long-step dual simplex will be used
+     7: mip =     not found yet >=              -inf        (1; 0)
Solution found by heuristic: 3339
+    10: mip =   3.339000000e+03 >=     tree is empty   0.0% (0; 7)
INTEGER OPTIMAL SOLUTION FOUND


,19,22,28,54,63,64,Resto
0,0,0,0,0,0,2,2
1,0,0,0,0,1,1,3
2,0,0,0,0,2,0,4
3,0,0,0,1,0,1,12
4,0,0,0,1,1,0,13
5,0,0,2,0,0,1,10
6,0,0,2,0,1,0,11
7,0,0,4,0,0,0,18
8,0,1,0,2,0,0,0
9,0,1,1,0,0,1,16


GLPK Simplex Optimizer 5.0
6 rows, 48 columns, 113 non-zeros
      0: obj =   0.000000000e+00 inf =   1.525e+04 (6)
      6: obj =   5.202175000e+03 inf =   0.000e+00 (0)
*    10: obj =   5.134708333e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer 5.0
6 rows, 48 columns, 113 non-zeros
48 integer variables, none of which are binary
Integer optimization begins...
Long-step dual simplex will be used
+    10: mip =     not found yet >=              -inf        (1; 0)
Solution found by heuristic: 5135
+    10: mip =   5.135000000e+03 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUND


,21,43,79,98,Resto
0,0,0,0,2,4
1,0,2,0,1,16
2,1,0,1,1,2
3,1,2,1,0,14
4,1,4,0,0,7
5,2,0,2,0,0
6,2,1,0,1,17
7,3,1,1,0,15
8,3,3,0,0,8
9,4,0,0,1,18


...

GLPK Simplex Optimizer 5.0
5 rows, 38 columns, 90 non-zeros
      0: obj =   0.000000000e+00 inf =   1.192e+04 (5)
      5: obj =   3.534675000e+03 inf =   0.000e+00 (0)
*     9: obj =   3.467208333e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer 5.0
5 rows, 38 columns, 90 non-zeros
38 integer variables, none of which are binary
Integer optimization begins...
Long-step dual simplex will be used
+     9: mip =     not found yet >=              -inf        (1; 0)
Solution found by heuristic: 3468
+     9: mip =   3.468000000e+03 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUND


,15,16,31,32,40,Resto
0,0,0,0,3,0,4
1,0,0,1,2,0,5
2,0,0,2,1,0,6
3,0,0,3,0,0,7
4,0,1,0,0,2,4
5,0,1,0,1,1,12
6,0,1,1,0,1,13
7,0,2,0,2,0,4
8,0,2,1,1,0,5
9,0,2,2,0,0,6


GLPK Simplex Optimizer 5.0
5 rows, 40 columns, 94 non-zeros
      0: obj =   0.000000000e+00 inf =   9.885e+03 (5)
      5: obj =   2.699166667e+03 inf =   0.000e+00 (0)
*     6: obj =   2.677500000e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer 5.0
5 rows, 40 columns, 94 non-zeros
40 integer variables, none of which are binary
Integer optimization begins...
Long-step dual simplex will be used
+     6: mip =     not found yet >=              -inf        (1; 0)
Solution found by heuristic: 2678
+     9: mip =   2.678000000e+03 >=     tree is empty   0.0% (0; 7)
INTEGER OPTIMAL SOLUTION FOUND


,33,52,60,87,Resto
0,0,0,1,2,16
1,0,0,4,0,10
2,0,1,0,2,24
3,0,1,3,0,18
4,0,2,2,0,26
5,0,3,0,1,7
6,1,0,2,1,10
7,1,1,1,1,18
8,1,2,0,1,26
9,1,3,1,0,1


GLPK Simplex Optimizer 5.0
4 rows, 25 columns, 58 non-zeros
      0: obj =   0.000000000e+00 inf =   9.932e+03 (4)
      4: obj =   2.449802632e+03 inf =   0.000e+00 (0)
*     5: obj =   2.447958333e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer 5.0
4 rows, 25 columns, 58 non-zeros
25 integer variables, none of which are binary
Integer optimization begins...
Long-step dual simplex will be used
+     5: mip =     not found yet >=              -inf        (1; 0)
+  1238: >>>>>   2.448000000e+03 >=   2.448000000e+03   0.0% (997; 0)
+  1238: mip =   2.448000000e+03 >=     tree is empty   0.0% (0; 1993)
INTEGER OPTIMAL SOLUTION FOUND
